# Assignment 2

**Credits**: Federico Ruggeri, Eleonora Mancini, Paolo Torroni

**Keywords**: Human Value Detection, Multi-label classification, Transformers, BERT


# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Federico Ruggeri -> federico.ruggeri6@unibo.it
* Eleonora Mancini -> e.mancini@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

# Introduction

You are tasked to address the [Human Value Detection challenge](https://aclanthology.org/2022.acl-long.306/).

## Problem definition

Arguments are paired with their conveyed human values.

Arguments are in the form of **premise** $\rightarrow$ **conclusion**.

### Example:

**Premise**: *``fast food should be banned because it is really bad for your health and is costly''*

**Conclusion**: *``We should ban fast food''*

**Stance**: *in favour of*

<center>
    <img src="images/human_values.png" alt="human values" />
</center>

# [Task 1 - 0.5 points] Corpus

Check the official page of the challenge [here](https://touche.webis.de/semeval23/touche23-web/).

The challenge offers several corpora for evaluation and testing.

You are going to work with the standard training, validation, and test splits.

#### Arguments
* arguments-training.tsv
* arguments-validation.tsv
* arguments-test.tsv

#### Human values
* labels-training.tsv
* labels-validation.tsv
* labels-test.tsv

### Example

#### arguments-*.tsv
```

Argument ID    A01005

Conclusion     We should ban fast food

Stance         in favor of

Premise        fast food should be banned because it is really bad for your health and is costly.
```

#### labels-*.tsv

```
Argument ID                A01005

Self-direction: thought    0
Self-direction: action     0
...
Universalism: objectivity: 0
```

### Splits

The standard splits contain

   * **Train**: 5393 arguments
   * **Validation**: 1896 arguments
   * **Test**: 1576 arguments

### Annotations

In this assignment, you are tasked to address a multi-label classification problem.

You are going to consider **level 3** categories:

* Openness to change
* Self-enhancement
* Conversation
* Self-transcendence

**How to do that?**

You have to merge (**logical OR**) annotations of level 2 categories belonging to the same level 3 category.

**Pay attention to shared level 2 categories** (e.g., Hedonism). $\rightarrow$ [see Table 1 in the original paper.](https://aclanthology.org/2022.acl-long.306/)

#### Example

```
Self-direction: thought:    0
Self-direction: action:     1
Stimulation:                0
Hedonism:                   1

Openess to change           1
```

### Instructions

* **Download** the specificed training, validation, and test files.
* **Encode** split files into a pandas.DataFrame object.
* For each split, **merge** the arguments and labels dataframes into a single dataframe.
* **Merge** level 2 annotations to level 3 categories.

In [131]:
# file management
import urllib
from pathlib import Path

# dataframe management
import pandas as pd

# data manipulation
import numpy as np

# for readability
from typing import Iterable
from tqdm import tqdm

#### Download

In [132]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(download_path: Path, url: str):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=download_path, reporthook=t.update_to)


def download_dataset(download_path: Path, url: str):
    print("Downloading dataset...")
    download_url(url=url, download_path=download_path)
    print("Download complete!")

def clean_download(url, name, folder):
    print(f"Current work directory: {Path.cwd()}")
    dataset_folder = Path.cwd().joinpath("Datasets").joinpath(folder)

    if not dataset_folder.exists():
        dataset_folder.mkdir(parents=True)

    dataset_path = dataset_folder.joinpath(name)

    if not dataset_path.exists():
        download_dataset(dataset_path, url)

In [133]:
arguments_split = ["arguments-training.tsv", "arguments-validation.tsv", "arguments-test.tsv"]
labels_split = ["labels-training.tsv", "labels-validation.tsv", "labels-test.tsv"]

for argument in arguments_split:
    url = "https://zenodo.org/records/8248658/files/" + argument + "?download=1"
    clean_download(url, argument, "Arguments")

for label in labels_split:
    url = "https://zenodo.org/records/8248658/files/" + label + "?download=1"
    clean_download(url, label, "Labels")


Current work directory: /Users/giacomopiergentili/Developer/NLP/Assignment2/NLP_multi-label-text-classification-with-transformers
Current work directory: /Users/giacomopiergentili/Developer/NLP/Assignment2/NLP_multi-label-text-classification-with-transformers
Current work directory: /Users/giacomopiergentili/Developer/NLP/Assignment2/NLP_multi-label-text-classification-with-transformers
Current work directory: /Users/giacomopiergentili/Developer/NLP/Assignment2/NLP_multi-label-text-classification-with-transformers
Current work directory: /Users/giacomopiergentili/Developer/NLP/Assignment2/NLP_multi-label-text-classification-with-transformers
Current work directory: /Users/giacomopiergentili/Developer/NLP/Assignment2/NLP_multi-label-text-classification-with-transformers


#### Encode

In [134]:
# Read the arguments split files into DataFrames
arguments_train_df = pd.read_table('Datasets/Arguments/arguments-training.tsv', sep='\t')
arguments_val_df = pd.read_table('Datasets/Arguments/arguments-validation.tsv', sep='\t')
arguments_test_df = pd.read_table('Datasets/Arguments/arguments-test.tsv', sep='\t')

# Read the labels split files into DataFrames
labels_train_df = pd.read_table('Datasets/Labels/labels-training.tsv', sep='\t')
labels_val_df = pd.read_table('Datasets/Labels/labels-validation.tsv', sep='\t')
labels_test_df = pd.read_table('Datasets/Labels/labels-test.tsv', sep='\t')


#### Merge arguments and labels

In [135]:
train_merge_df = pd.merge(arguments_train_df, labels_train_df, on='Argument ID')
val_merge_df = pd.merge(arguments_val_df, labels_val_df, on='Argument ID')
test_merge_df = pd.merge(arguments_test_df, labels_test_df, on='Argument ID')


#### Merge level 2 annotations to level 3 categories

* Openness to change
* Self-enhancement
* Conversation
* Self-transcendence

In [136]:
level_2_categories = train_merge_df.columns[4:]

In [137]:
level_2_to_Openness_to_change = level_2_categories[:4]
level_2_to_Self_enhancement = level_2_categories[3:8]
level_2_to_Conservation = level_2_categories[7:14]
level_2_to_Self_transcendence = level_2_categories[13:]

In [138]:
def merge_categories(df):
    df['Openness to change'] = [int(any(df[level_2_to_Openness_to_change].loc[i])) for i in range(len(df))]
    df['Self-enhancement'] = [int(any(df[level_2_to_Self_enhancement].loc[i])) for i in range(len(df))]
    df['Conservation'] = [int(any(df[level_2_to_Conservation].loc[i])) for i in range(len(df))]
    df['Self-trascendence'] = [int(any(df[level_2_to_Self_transcendence].loc[i])) for i in range(len(df))]
    return df.drop(level_2_categories, axis=1)

In [139]:
final_train_df = merge_categories(train_merge_df)
final_val_df = merge_categories(val_merge_df)
final_test_df = merge_categories(test_merge_df)

# [Task 2 - 2.0 points] Model definition

You are tasked to define several neural models for multi-label classification.

<center>
    <img src="images/model_schema.png" alt="model_schema" />
</center>

### Instructions

* **Baseline**: implement a random uniform classifier (an individual classifier per category).
* **Baseline**: implement a majority classifier (an individual classifier per category).

<br/>

* **BERT w/ C**: define a BERT-based classifier that receives an argument **conclusion** as input.
* **BERT w/ CP**: add argument **premise** as an additional input.
* **BERT w/ CPS**: add argument premise-to-conclusion **stance** as an additional input.

### Notes

**Do not mix models**. Each model has its own instructions.

You are **free** to select the BERT-based model card from huggingface.

#### Examples

```
bert-base-uncased
prajjwal1/bert-tiny
distilbert-base-uncased
roberta-base
```

### BERT w/ C

<center>
    <img src="images/bert_c.png" alt="BERT w/ C" />
</center>

### BERT w/ CP

<center>
    <img src="images/bert_cp.png" alt="BERT w/ CP" />
</center>

### BERT w/ CPS

<center>
    <img src="images/bert_cps.png" alt="BERT w/ CPS" />
</center>

### Input concatenation

<center>
    <img src="images/input_merging.png" alt="Input merging" />
</center>

### Notes

The **stance** input has to be encoded into a numerical format.

You **should** use the same model instance to encode **premise** and **conclusion** inputs.

# [Task 3 - 0.5 points] Metrics

Before training the models, you are tasked to define the evaluation metrics for comparison.

### Instructions

* Evaluate your models using per-category binary F1-score.
* Compute the average binary F1-score over all categories (macro F1-score).

### Example

You start with individual predictions ($\rightarrow$ samples).

```
Openess to change:    0 0 1 0 1 1 0 ...
Self-enhancement:     1 0 0 0 1 0 1 ...
Conversation:         0 0 0 1 1 0 1 ...
Self-transcendence:   1 1 0 1 0 1 0 ...
```

You compute per-category binary F1-score.

```
Openess to change F1:    0.35
Self-enhancement F1:     0.55
Conversation F1:         0.80
Self-transcendence F1:   0.21
```

You then average per-category scores.
```
Average F1: ~0.48
```

# [Task 4 - 1.0 points] Training and Evaluation

You are now tasked to train and evaluate **all** defined models.

### Instructions

* Train **all** models on the train set.
* Evaluate **all** models on the validation set.
* Pick **at least** three seeds for robust estimation.
* Compute metrics on the validation set.
* Report **per-category** and **macro** F1-score for comparison.

# [Task 5 - 1.0 points] Error Analysis

You are tasked to discuss your results.

### Instructions

* **Compare** classification performance of BERT-based models with respect to baselines.
* Discuss **difference in prediction** between the best performing BERT-based model and its variants.

### Notes

You can check the [original paper](https://aclanthology.org/2022.acl-long.306/) for suggestions on how to perform comparisons (e.g., plots, tables, etc...).

# [Task 6 - 1.0 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...
* You can upload **model weights** in a cloud repository and report the link in the report.

# FAQ

Please check this frequently asked questions before contacting us

### Model card

You are **free** to choose the BERT-base model card you like from huggingface.

### Model architecture

You **should not** change the architecture of a model (i.e., its layers).

However, you are **free** to play with their hyper-parameters.

### Model Training

You are **free** to choose training hyper-parameters for BERT-based models (e.g., number of epochs, etc...).

### Neural Libraries

You are **free** to use any library of your choice to address the assignment (e.g., Keras, Tensorflow, PyTorch, JAX, etc...)

### Error Analysis

Some topics for discussion include:
   * Model performance on most/less frequent classes.
   * Precision/Recall curves.
   * Confusion matrices.
   * Specific misclassified samples.

# The End